#### DeltaReader

In [ ]:
import scala.io.Source
import scala.util.{Try, Using}

object DeltaReader {

  // @param - optional, tableName Name of Master delta table to read, it will retrieve map by pipeline name by default
  // @return DataFrame with correct (baseline or incremental) data
  def readDeltaWithWatermark(
    configPath: String, 
    tableNames: Seq[String] = Seq.empty
  ): DataFrame = {

    val versionMap = Watermarks.getWatermarkForTable(tableNames)
    
    // BASELINE MODE: full read at version N
    if (startVersion == endVersion) {
      println(s"[Baseline] Reading full table `$tableNames` at version $endVersion")
      spark.read
        .format("delta")
        .option("versionAsOf", endVersion)
        .table(tableName)
    }
    // INCREMENTAL MODE: read only deltas M -> N
    else {
      println(s"[Incremental] Reading delta from version $versionMap for `$tableNames`")

      val yaml = readResourceAsString(configPath)
      // Generic logic to parse YAML → SQL string or version map
      val sql = QueryProcessor.buidlSQL(yaml, versionMap)
      spark.sql(sql)  

      /*
      spark.read
        .format("delta")
        .option("startingVersion", startVersion)
        .option("endingVersion", endVersion)
        .table(tableName)
       */
    }
  }

  private def readResourceAsString(
    resourcePath: String
  ): Try[String] = {

    Using.resource(Source.fromResource(resourcePath)) { source =>
      source.mkString
    }
}

object DeltaWriter {

  def writeDelta(
    df: DataFrame, 
    targetTable: String
  ): Unit = {
    
    df.write.format("delta")
      .mode("mergeUpdate") // or append/upsert
      .saveAsTable(targetTable)
  }
}
